In [ ]:
docker run -it --entrypoint bash python:3.13
pip --version

25.3

In [1]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet

zsh:1: command not found: wget


In [2]:
pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 2.5 MB/s  0:00:05 eta 0:00:01m

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
!curl -L -o green_tripdata_2025-11.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1137k  100 1137k    0     0   262k      0  0:00:04  0:00:04 --:--:--  262k


In [14]:
!curl -L -o taxi_zone_lookup.csv https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12322  100 12322    0     0  14738      0 --:--:-- --:--:-- --:--:-- 14738


In [10]:
import duckdb

con = duckdb.connect()

result = con.execute("""
SELECT COUNT(*)
FROM 'green_tripdata_2025-11.parquet'
WHERE lpep_pickup_datetime >= '2025-11-01'
  AND lpep_pickup_datetime <  '2025-12-01'
  AND trip_distance <= 1
""").fetchone()

print(result)

(8007,)


In [15]:
import duckdb

con = duckdb.connect()

con.execute("""
DESCRIBE
SELECT *
FROM 'green_tripdata_2025-11.parquet'
""").df()

,column_name,column_type,null,key,default,extra
0,VendorID,INTEGER,YES,None,None,None
1,lpep_pickup_datetime,TIMESTAMP,YES,None,None,None
2,lpep_dropoff_datetime,TIMESTAMP,YES,None,None,None
3,store_and_fwd_flag,VARCHAR,YES,None,None,None
4,RatecodeID,BIGINT,YES,None,None,None
5,PULocationID,INTEGER,YES,None,None,None
6,DOLocationID,INTEGER,YES,None,None,None
7,passenger_count,BIGINT,YES,None,None,None
8,trip_distance,DOUBLE,YES,None,None,None
9,fare_amount,DOUBLE,YES,None,None,None


In [39]:
import duckdb

con = duckdb.connect()

result = con.execute("""
SELECT SUM(VendorID) AS total_vendor_id, taxi.Zone
FROM 'green_tripdata_2025-11.parquet'
JOIN 'taxi_zone_lookup.csv' taxi
ON PULocationID = LocationID
                     GROUP By taxi.Zone
                     ORDER BY total_vendor_id DESC
                     LIMIT 1
""").fetchall()

print(result)

[(23279, 'East Harlem North')]


In [46]:
import duckdb

con = duckdb.connect()

result = con.execute("""
SELECT
    MAX(tip_amount) AS max_tip, z.Zone  
FROM 'green_tripdata_2025-11.parquet' t
JOIN 'taxi_zone_lookup.csv' z
  ON t.PULocationID = z.LocationID
WHERE t.lpep_pickup_datetime >= '2025-11-01'
  AND t.lpep_pickup_datetime <  '2025-12-01'
  AND z.Zone = 'East Harlem North'
GROUP BY z.Zone
""").fetchone()

print(result)

(81.89, 'East Harlem North')


In [ ]:
print("end")